In [ ]:
%matplotlib inline
import bz2
import json
import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout

In [ ]:
data = json.load(bz2.BZ2File("testdata/pythia6_ttbar_0001_pandora_0.json.bz2", "r"))

In [ ]:
#http://flc.desy.de/lcnotes/notes/localfsExplorer_read?currentPath=/afs/desy.de/group/flc/lcnotes/LC-DET-2006-004.pdf 
a = 3*10**-4
b = 5 #B-field in tesla 

In [ ]:
def track_pt(omega):
    return a*np.abs(b/omega)

In [ ]:
iev = 3
df_gen = pandas.DataFrame(data[iev]["genparticles"])

df_hit = pandas.DataFrame(data[iev]["track_hits"])
df_cl = pandas.DataFrame(data[iev]["clusters"])
df_tr = pandas.DataFrame(data[iev]["tracks"])
df_ecal = pandas.DataFrame(data[iev]["ecal_hits"])
df_hcal = pandas.DataFrame(data[iev]["hcal_hits"])
df_pfs = pandas.DataFrame(data[iev]["pfs"])

df_tr["pt"] = track_pt(df_tr["omega"])
df_tr["px"] = np.cos(df_tr["phi"])*df_tr["pt"]
df_tr["py"] = np.sin(df_tr["phi"])*df_tr["pt"]
df_tr["pz"] = df_tr["tan_lambda"]*df_tr["pt"]

In [ ]:
df_gen[df_gen["pdgid"].abs()==15]

In [ ]:
df_hit

In [ ]:
df_ecal

In [ ]:
df_hcal

In [ ]:
df_tr

In [ ]:
df_cl

In [ ]:
df_gen

In [ ]:
df_pfs

In [ ]:
g = nx.DiGraph()
for igen in range(len(df_gen)):
    g.add_node("gen{}".format(igen), typ=int(df_gen.iloc[igen]["pdgid"]), e=df_gen.iloc[igen]["energy"])
    
for igen in range(len(df_gen)):
    idx_parent0 = int(df_gen.iloc[igen]["idx_parent0"])
    if idx_parent0 != -1:
        g.add_edge("gen{}".format(idx_parent0), "gen{}".format(igen))
        
for icl in range(len(df_cl)):
    g.add_node("clu{}".format(icl), typ=df_cl.iloc[icl]["type"], e=df_cl.iloc[icl]["energy"])
    for gp, gp_w in df_cl.iloc[icl]["gp_contributions"].items():
        if gp_w/df_cl.iloc[icl]["energy"]>0.2:
            g.add_edge("gen{}".format(gp), "clu{}".format(icl))

for itr in range(len(df_tr)):
    g.add_node("tra{}".format(itr), typ=0, e=df_tr.iloc[itr]["pt"])
    for gp, gp_w in df_tr.iloc[itr]["gp_contributions"].items():
        if gp_w/df_tr.iloc[itr]["nhits"]>0.2:
            g.add_edge("gen{}".format(gp), "tra{}".format(itr))
        
for ipf in range(len(df_pfs)):
    g.add_node("pfo{}".format(ipf), typ=int(df_pfs.iloc[ipf]["type"]), e=df_pfs.iloc[ipf]["energy"])
    cl_idx = int(df_pfs.iloc[ipf]["cluster_idx"])
    if cl_idx!=-1:
        g.add_edge("clu{}".format(cl_idx), "pfo{}".format(ipf))

    tr_idx = int(df_pfs.iloc[ipf]["track_idx"])
    if tr_idx!=-1:
        g.add_edge("tra{}".format(tr_idx), "pfo{}".format(ipf))

In [ ]:
def node_color(node):
    if node.startswith("gen"):
        if abs(g.nodes[node]["typ"])==15:
            return "purple"
        return "red"
    elif node.startswith("clu"):
        return "blue"
    elif node.startswith("tra"):
        return "green"
    else:
        return "gray"
    
def node_label(node):
    typ = node[:4]
    l = "{}".format(g.nodes[node]["typ"])
    return l

In [ ]:
plt.figure(figsize=(50,30))
pos = graphviz_layout(g, prog="dot")
nx.draw_networkx_nodes(g, pos,
    node_size=[5*g.nodes[n]["e"] for n in g.nodes],
    node_color=[node_color(n) for n in g.nodes],
)
nx.draw_networkx_labels(g, pos,
    labels={n: node_label(n) for n in g.nodes},
    font_size=5
)
nx.draw_networkx_edges(g, pos, node_size=100.0);
plt.savefig("plot.svg")